In [ ]:
import os
import sys
sys.path.append("..")
import canon

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import tensorflow as tf
import keras, keras.layers as L, keras.backend as K

from canon.autoencode.feeder import ImageDataFeeder

IMAGE_DIR = "img/processed_128"
filenames = [filename for filename in os.listdir(IMAGE_DIR)]
IMAGE_SHAPE = plt.imread(os.path.join(IMAGE_DIR, filenames[0])).shape
feeder = ImageDataFeeder(batch_size=30, test_size=500, img_dir=IMAGE_DIR)
X_test = feeder.get_test_set()
print(X_test.shape)

In [ ]:
import canon.autoencode
from canon.autoencode import reset_tf_session

s = reset_tf_session()
model_name = canon.autoencode.models.AE_128_to_256
encoder = canon.autoencode.models.load_encoder(model_name)
decoder = canon.autoencode.models.load_decoder(model_name)
encoder.summary()
decoder.summary()

In [ ]:
def visualize(img,encoder,decoder):
    """Draws original, encoded and decoded images"""
    code = encoder.predict(img[None])[0]  # img[None] is the same as img[np.newaxis, :]
    reco = decoder.predict(code[None])[0]

    plt.figure(figsize=(10,50))
    
    plt.subplot(1,3,1)
    plt.title("Original")
    plt.imshow(img, cmap='gray')

    plt.subplot(1,3,2)
    plt.title("Code")
    plt.imshow(code.reshape([int(np.ceil(np.sqrt(code.shape[-1]))),-1]), cmap='gray')

    plt.subplot(1,3,3)
    plt.title("Reconstructed")
    plt.imshow(reco, cmap='gray')
    plt.show()

for i in np.random.choice(np.arange(20), 5):
    img = X_test[i]
    visualize(img, encoder, decoder)